<a href="https://colab.research.google.com/github/codingFerryman/crypto_market_hierarchy_structure/blob/main/src/crypto_correlation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
import pandas as pd
import numpy as np
from scipy.signal import correlate
from scipy.stats import spearmanr
from scipy.stats import pearsonr
import glob
import os

###Preprocessing and Load data

+ Preprocessing missing value

+ Load data

+ Build up list of original data vectors

+ Get the name list of cryptocurrency

#### Load data, Build up list of original data vectors, Get the name list of cryptocurrency

In [6]:
# Get file paths
def get_file_path(folder_path):
  paths = glob.glob(os.path.join(folder_path, "*"))
  paths = sorted(paths)
  return paths

In [7]:
#test code
paths = get_file_path("data/3h")
paths

['data/3h/1INCH_USD_3h.csv',
 'data/3h/AAVE_USD_3h.csv',
 'data/3h/ADA_USD_3h.csv']

假装拿到process好的数据文件， 按interval读

In [85]:
# create the list of crytocurrencies price records, 
# each item means the list of original price vectors of each cryptocurrency
# each vector contains all the close prices of per day / per week (determing with time scale)
def get_interval(timescale):
  if timescale == '3h':
    return 8
  if timescale == '30m':
    return 1

def build_data_list(paths, timescale):
  interval = get_interval(timescale)
  
  data_list = []
  name_list = []
  
  for i in range(0, len(paths)):
    idx = 0
    shift_idx = interval - 1
    cryptocurrency_price_list = []
    cryptocurrency_df = pd.read_csv(paths[i])
    # discard the last row
    cryptocurrency_df = cryptocurrency_df.iloc[:-1 , :]
    
    #record the name of crytocurrency
    name_list.append(cryptocurrency_df["coin"][0])

    #discard other columns except close price
    close_price_df = cryptocurrency_df["close"]
  
    while shift_idx <= len(close_price_df):
      vector = close_price_df.loc[idx:shift_idx]
      #Calculate the price ratio of changing
      vector = vector.to_frame()
      vector = vector.pct_change().fillna(0)

      cryptocurrency_price_list.append(np.asarray(vector))
      idx += interval
      shift_idx += interval

    print(len(cryptocurrency_price_list))
    data_list.append(cryptocurrency_price_list)  

  return data_list, name_list

data_list, name_list = build_data_list(paths, '3h')

63
63
64


###Define functions to calculate correlation matrix

+ Pearson correlation coefficient
+ Spearman rank-order correlation coefficient

In [49]:
# Calculate correlation matrix - Pearson correlation coefficient
def cm_pearson(x, y):
  pccs = pearsonr(x,y)
  return pccs[0]

# Calculate correlation matrix - Spearman rank-order correlation coefficient
def cm_spearman(x, y):
  sccs = spearmanr(x, y)
  return sccs[0]

###Calculate Correlation matrix

In [71]:
def calculate_cm(data_list, func):
    N = len(data_list)
    cm_res = np.zeros((N,N))
    for day in range(0, len(data_list[0])):
        for i in range(N):
          for j in range(N):
            data_list[i][day] = data_list[i][day].squeeze()
            data_list[j][day] = data_list[j][day].squeeze()
            res = func(data_list[i][day], data_list[j][day]) 
            cm_res[i,j] = res

    return cm_res

In [88]:
res = calculate_cm(data_list, cm_pearson)
print(res.shape)
print(res)

(3, 3)
[[1.         0.6417141  0.20744414]
 [0.6417141  1.         0.36042915]
 [0.20744414 0.36042915 1.        ]]


### Save correlation matrix to files in format of Numpy array